In [2]:
# Woahhhh, hey everyone Cass here. I used SO MUCH F***ING AI HERE, so yea, idfk so much of this. 

# Importing libraries
import medspacy
import spacy
from spacy.tokens import Doc
import torch
import numpy
from transformers import AutoTokenizer, AutoModel

# Loading the Bio_ClinicalBERT model and tokenizer
# The first option below is not ideal, as this is downloading from HuggingFace each time. The below can be ran if you don't have the model downloaded.
# You may need HuggingFace_Hub to download the model and tokenizer to your local machine, and then load them from there. This is what I did.

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

print("imported relevant libraries)")

imported relevant libraries)


In [3]:


# Load a medspacy (spaCy) pipeline and replace its tokenizer with one that uses the HF tokenizer offsets
nlp = medspacy.load()

# Defines a factory function replacing the default spaCy tokenizer. This ensures that the tokenization is consistent with the Bio_ClinicalBERT model's expectations.
def hf_spacy_tokenizer_factory(nlp):

    # Define function to create custom tokeniser using Hugging Face tokenizer's offsets to (1) create tokens and (2) determine if a space follows each token
    def tokenize(text):
        enc = tokenizer(text, return_offsets_mapping=True, add_special_tokens=False)
        offsets = enc["offset_mapping"]
        # Build token texts from offsets and whether a space follows
        words = [text[s:e] for s, e in offsets]
        spaces = []
        for s, e in offsets:
            spaces.append(e < len(text) and text[e].isspace())
        return Doc(nlp.vocab, words=words, spaces=spaces)
    
    return tokenize

nlp.tokenizer = hf_spacy_tokenizer_factory(nlp)

print("Model and tokenizer loaded, and spaCy pipeline configured with custom tokenizer.")


Model and tokenizer loaded, and spaCy pipeline configured with custom tokenizer.


In [4]:
# Read complaints and run pipeline
with open("complaints2.txt", "r", encoding="utf-8") as f:
    complainttext = f.read()

nlp.max_length = len(complainttext) + 1000  # Set max_length to be slightly larger than the complaint text length
doc = nlp(complainttext)

2026-02-07 23:11:30.635 | DEBUG    | PyRuSH.PyRuSHSentencizer:predict:100 - [cpredict_split_gaps|call_id=0] [doc 0] Token 0 'P' marked as sentence start (span begin)
2026-02-07 23:11:30.638 | DEBUG    | PyRuSH.PyRuSHSentencizer:predict:100 - [cpredict_split_gaps|call_id=0] Token/tag mapping: [(P, True), (IC, False), (C, False), (E, False), (VAL, False), (,, False), (FACIAL, False), (D, False), (ROO, False), (P, False), (,, False), (M, False), (V, False), (C, False), (/, False), (IN, False), (TUB, False), (ATED, False), (TRAUMA, False), (,, False), (S, False), (/, False), (P, False), (ARREST, False), (,, False), (HEAD, False), (BLEED, False), (,, False), (SOB, False), (,, False), (UNKNOWN, False), (-, False), (CC, False), (,, False), (HEAD, False), (BLEED, False), (,, False), (HEAD, False), (BLEED, False), (,, False), (SEIZURE, False), (,, False), (RE, False), (SP, False), (ARREST, False), (,, False), (S, False), (W, False), (,, False), (CAR, False), (VS, False), (POLE, False), (,, Fals

In [5]:
# Show tokens
print("First 50 tokens:")
for token in doc[:50]:
    print(token.i, token.text)

First 50 tokens:
0 P
1 IC
2 C
3 E
4 VAL
5 ,
6 FACIAL
7 D
8 ROO
9 P
10 ,
11 M
12 V
13 C
14 /
15 IN
16 TUB
17 ATED
18 TRAUMA
19 ,
20 S
21 /
22 P
23 ARREST
24 ,
25 HEAD
26 BLEED
27 ,
28 SOB
29 ,
30 UNKNOWN
31 -
32 CC
33 ,
34 HEAD
35 BLEED
36 ,
37 HEAD
38 BLEED
39 ,
40 SEIZURE
41 ,
42 RE
43 SP
44 ARREST
45 ,
46 S
47 W
48 ,
49 CAR
